In [71]:
# Importing the required libraries
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import hvplot
import hvplot.pandas
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [72]:
# Importing the insider data from CSV
insider_df = pd.read_csv("insider_data_v2.csv",index_col="Filing Date", parse_dates=True, infer_datetime_format=True)
insider_df.index = insider_df.index.date

In [73]:
# Displaying the DataFrame
display(insider_df.head())
display(insider_df.tail())

,Trade Date,X,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value
2023-06-05,2023-06-02,D,MSFT,List Teri,Dir,S - Sale+OE,$334.77,"-1,250","1,282",-49%,"-$418,463"
2023-05-19,2023-05-18,NaN,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,$316.87,"-5,000","90,632",-5%,"-$1,584,354"
2023-05-17,2023-05-16,NaN,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,$312.54,"-5,081","95,632",-5%,"-$1,588,024"
2023-05-10,2023-05-09,M,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,$310.62,"-9,177","102,333",-8%,"-$2,850,529"
2023-05-08,2023-05-05,NaN,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,$310.05,"-5,000","111,510",-4%,"-$1,550,250"


,Trade Date,X,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value
2019-09-19,2019-09-17,NaN,TSLA,Kirkhorn Zachary,CFO,S - Sale,$242.47,-150,"10,001",-1%,"-$36,371"
2019-08-21,2019-08-19,NaN,TSLA,Kirkhorn Zachary,CFO,S - Sale,$224.00,-150,"9,812",-2%,"-$33,600"
2019-07-31,2019-07-29,NaN,TSLA,Straubel Jeffrey B,Chief Technical Officer,S - Sale,$230.45,"-15,000","276,398",-5%,"-$3,456,732"
2019-07-19,2019-07-17,NaN,TSLA,Kirkhorn Zachary,CFO,S - Sale,$255.68,-150,"9,962",-1%,"-$38,352"
2019-07-02,2019-06-28,NaN,TSLA,Straubel Jeffrey B,Chief Technical Officer,S - Sale,$223.51,"-15,000","291,398",-5%,"-$3,352,630"


In [74]:
# Replace the commas and $ values from the data columns

insider_df["Price"] = insider_df["Price"].str.replace("$", "")
insider_df["Price"] = insider_df["Price"].str.replace(",", "")
insider_df["Qty"] = insider_df["Qty"].str.replace(",", "")
insider_df["Owned"] = insider_df["Owned"].str.replace(",", "")
insider_df["Value"] = insider_df["Value"].str.replace("$", "")
insider_df["Value"] = insider_df["Value"].str.replace(",", "")

# Drop column X
insider_df = insider_df.drop(columns="X")

display(insider_df.head(10))
display(insider_df.tail(10))

/var/folders/kb/xkf419255xx2rp_mmzmhfwsh0000gn/T/ipykernel_51726/215898364.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  insider_df["Price"] = insider_df["Price"].str.replace("$", "")
/var/folders/kb/xkf419255xx2rp_mmzmhfwsh0000gn/T/ipykernel_51726/215898364.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  insider_df["Value"] = insider_df["Value"].str.replace("$", "")


,Trade Date,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value
2023-06-05,2023-06-02,MSFT,List Teri,Dir,S - Sale+OE,334.77,-1250,1282,-49%,-418463
2023-05-19,2023-05-18,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,316.87,-5000,90632,-5%,-1584354
2023-05-17,2023-05-16,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,312.54,-5081,95632,-5%,-1588024
2023-05-10,2023-05-09,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,310.62,-9177,102333,-8%,-2850529
2023-05-08,2023-05-05,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,310.05,-5000,111510,-4%,-1550250
2023-05-02,2023-05-01,MSFT,Althoff Judson,"EVP, Chief Commercial Officer",S - Sale,306.09,-30000,117456,-20%,-9182700
2023-03-02,2023-02-28,MSFT,Nadella Satya,CEO,S - Sale,248.88,-4767,788625,-1%,-1186411
2023-02-15,2023-02-13,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,272.32,-1000,117666,-1%,-272323
2023-02-13,2023-02-09,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,269.38,-1000,118666,-1%,-269380
2022-12-02,2022-12-01,MSFT,Althoff Judson,"EVP, Chief Commercial Officer",S - Sale,254.27,-24144,150047,-14%,-6139124


,Trade Date,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value
2019-11-14,2019-11-12,TSLA,Musk Kimbal,Dir,S - Sale+OE,350.06,-64285,134048,-32%,-22503768
2019-11-05,2019-11-01,TSLA,Guillen Jerome M,"Pres, Automotive",S - Sale+OE,316.32,-2000,8973,-18%,-632640
2019-11-01,2019-10-30,TSLA,Baglino Andrew D,"SVP Powertrain, Energy Eng.",S - Sale+OE,313.25,-2366,3288,-42%,-741159
2019-10-29,2019-10-25,TSLA,Guillen Jerome M,"Pres, Automotive",S - Sale+OE,305.00,-8000,8973,-47%,-2440000
2019-10-21,2019-10-17,TSLA,Kirkhorn Zachary,CFO,S - Sale,262.00,-150,9851,-1%,-39300
2019-09-19,2019-09-17,TSLA,Kirkhorn Zachary,CFO,S - Sale,242.47,-150,10001,-1%,-36371
2019-08-21,2019-08-19,TSLA,Kirkhorn Zachary,CFO,S - Sale,224.00,-150,9812,-2%,-33600
2019-07-31,2019-07-29,TSLA,Straubel Jeffrey B,Chief Technical Officer,S - Sale,230.45,-15000,276398,-5%,-3456732
2019-07-19,2019-07-17,TSLA,Kirkhorn Zachary,CFO,S - Sale,255.68,-150,9962,-1%,-38352
2019-07-02,2019-06-28,TSLA,Straubel Jeffrey B,Chief Technical Officer,S - Sale,223.51,-15000,291398,-5%,-3352630


In [75]:
# Getting details on the coluns and the data
insider_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1509 entries, 2023-06-05 to 2019-07-02
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Trade Date    1509 non-null   object
 1   Ticker        1509 non-null   object
 2   Insider Name  1509 non-null   object
 3   Title         1509 non-null   object
 4   Trade Type    1509 non-null   object
 5   Price         1509 non-null   object
 6   Qty           1509 non-null   object
 7   Owned         1509 non-null   object
 8   ΔOwn          1509 non-null   object
 9   Value         1509 non-null   object
dtypes: object(10)
memory usage: 129.7+ KB


In [76]:
# Convert columns to float
insider_df["Qty"] = insider_df["Qty"].astype('float')
insider_df["Price"] = insider_df["Price"].astype('float')
insider_df["Owned"] = insider_df["Owned"].astype('float')
insider_df["Value"] = insider_df["Value"].astype('float')

insider_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1509 entries, 2023-06-05 to 2019-07-02
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Trade Date    1509 non-null   object 
 1   Ticker        1509 non-null   object 
 2   Insider Name  1509 non-null   object 
 3   Title         1509 non-null   object 
 4   Trade Type    1509 non-null   object 
 5   Price         1509 non-null   float64
 6   Qty           1509 non-null   float64
 7   Owned         1509 non-null   float64
 8   ΔOwn          1509 non-null   object 
 9   Value         1509 non-null   float64
dtypes: float64(4), object(6)
memory usage: 129.7+ KB


In [77]:

# Convert negative numbers to positve as the data only includes sales
negative_numbers_qty = insider_df["Qty"] < 0
negative_numbers_price = insider_df["Price"] < 0
negative_numbers_value = insider_df["Value"] < 0

insider_df["Qty_Pos"] = insider_df["Qty"].where(~negative_numbers_qty, -insider_df["Qty"])
insider_df["Price_Pos"] = insider_df["Price"].where(~negative_numbers_price, -insider_df["Price"])
insider_df["Value_Pos"] = insider_df["Value"].where(~negative_numbers_value, -insider_df["Value"])

# Display the dateframe
display(insider_df.head(10))
display(insider_df.tail(10))



,Trade Date,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value,Qty_Pos,Price_Pos,Value_Pos
2023-06-05,2023-06-02,MSFT,List Teri,Dir,S - Sale+OE,334.77,-1250.0,1282.0,-49%,-418463.0,1250.0,334.77,418463.0
2023-05-19,2023-05-18,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,316.87,-5000.0,90632.0,-5%,-1584354.0,5000.0,316.87,1584354.0
2023-05-17,2023-05-16,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,312.54,-5081.0,95632.0,-5%,-1588024.0,5081.0,312.54,1588024.0
2023-05-10,2023-05-09,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,310.62,-9177.0,102333.0,-8%,-2850529.0,9177.0,310.62,2850529.0
2023-05-08,2023-05-05,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,310.05,-5000.0,111510.0,-4%,-1550250.0,5000.0,310.05,1550250.0
2023-05-02,2023-05-01,MSFT,Althoff Judson,"EVP, Chief Commercial Officer",S - Sale,306.09,-30000.0,117456.0,-20%,-9182700.0,30000.0,306.09,9182700.0
2023-03-02,2023-02-28,MSFT,Nadella Satya,CEO,S - Sale,248.88,-4767.0,788625.0,-1%,-1186411.0,4767.0,248.88,1186411.0
2023-02-15,2023-02-13,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,272.32,-1000.0,117666.0,-1%,-272323.0,1000.0,272.32,272323.0
2023-02-13,2023-02-09,MSFT,Capossela Christopher C,"EVP, CMO",S - Sale,269.38,-1000.0,118666.0,-1%,-269380.0,1000.0,269.38,269380.0
2022-12-02,2022-12-01,MSFT,Althoff Judson,"EVP, Chief Commercial Officer",S - Sale,254.27,-24144.0,150047.0,-14%,-6139124.0,24144.0,254.27,6139124.0


,Trade Date,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value,Qty_Pos,Price_Pos,Value_Pos
2019-11-14,2019-11-12,TSLA,Musk Kimbal,Dir,S - Sale+OE,350.06,-64285.0,134048.0,-32%,-22503768.0,64285.0,350.06,22503768.0
2019-11-05,2019-11-01,TSLA,Guillen Jerome M,"Pres, Automotive",S - Sale+OE,316.32,-2000.0,8973.0,-18%,-632640.0,2000.0,316.32,632640.0
2019-11-01,2019-10-30,TSLA,Baglino Andrew D,"SVP Powertrain, Energy Eng.",S - Sale+OE,313.25,-2366.0,3288.0,-42%,-741159.0,2366.0,313.25,741159.0
2019-10-29,2019-10-25,TSLA,Guillen Jerome M,"Pres, Automotive",S - Sale+OE,305.00,-8000.0,8973.0,-47%,-2440000.0,8000.0,305.00,2440000.0
2019-10-21,2019-10-17,TSLA,Kirkhorn Zachary,CFO,S - Sale,262.00,-150.0,9851.0,-1%,-39300.0,150.0,262.00,39300.0
2019-09-19,2019-09-17,TSLA,Kirkhorn Zachary,CFO,S - Sale,242.47,-150.0,10001.0,-1%,-36371.0,150.0,242.47,36371.0
2019-08-21,2019-08-19,TSLA,Kirkhorn Zachary,CFO,S - Sale,224.00,-150.0,9812.0,-2%,-33600.0,150.0,224.00,33600.0
2019-07-31,2019-07-29,TSLA,Straubel Jeffrey B,Chief Technical Officer,S - Sale,230.45,-15000.0,276398.0,-5%,-3456732.0,15000.0,230.45,3456732.0
2019-07-19,2019-07-17,TSLA,Kirkhorn Zachary,CFO,S - Sale,255.68,-150.0,9962.0,-1%,-38352.0,150.0,255.68,38352.0
2019-07-02,2019-06-28,TSLA,Straubel Jeffrey B,Chief Technical Officer,S - Sale,223.51,-15000.0,291398.0,-5%,-3352630.0,15000.0,223.51,3352630.0


In [78]:
# Verifying the column attributes
insider_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1509 entries, 2023-06-05 to 2019-07-02
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Trade Date    1509 non-null   object 
 1   Ticker        1509 non-null   object 
 2   Insider Name  1509 non-null   object 
 3   Title         1509 non-null   object 
 4   Trade Type    1509 non-null   object 
 5   Price         1509 non-null   float64
 6   Qty           1509 non-null   float64
 7   Owned         1509 non-null   float64
 8   ΔOwn          1509 non-null   object 
 9   Value         1509 non-null   float64
 10  Qty_Pos       1509 non-null   float64
 11  Price_Pos     1509 non-null   float64
 12  Value_Pos     1509 non-null   float64
dtypes: float64(7), object(6)
memory usage: 165.0+ KB


In [79]:
insider_df.isna().sum()

Trade Date      0
Ticker          0
Insider Name    0
Title           0
Trade Type      0
Price           0
Qty             0
Owned           0
ΔOwn            0
Value           0
Qty_Pos         0
Price_Pos       0
Value_Pos       0
dtype: int64

In [80]:
# Change the Trade Type to S
insider_df["Trade Type"] = "s"
display(insider_df)

,Trade Date,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value,Qty_Pos,Price_Pos,Value_Pos
2023-06-05,2023-06-02,MSFT,List Teri,Dir,s,334.77,-1250.0,1282.0,-49%,-418463.0,1250.0,334.77,418463.0
2023-05-19,2023-05-18,MSFT,Capossela Christopher C,"EVP, CMO",s,316.87,-5000.0,90632.0,-5%,-1584354.0,5000.0,316.87,1584354.0
2023-05-17,2023-05-16,MSFT,Capossela Christopher C,"EVP, CMO",s,312.54,-5081.0,95632.0,-5%,-1588024.0,5081.0,312.54,1588024.0
2023-05-10,2023-05-09,MSFT,Capossela Christopher C,"EVP, CMO",s,310.62,-9177.0,102333.0,-8%,-2850529.0,9177.0,310.62,2850529.0
2023-05-08,2023-05-05,MSFT,Capossela Christopher C,"EVP, CMO",s,310.05,-5000.0,111510.0,-4%,-1550250.0,5000.0,310.05,1550250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-19,2019-09-17,TSLA,Kirkhorn Zachary,CFO,s,242.47,-150.0,10001.0,-1%,-36371.0,150.0,242.47,36371.0
2019-08-21,2019-08-19,TSLA,Kirkhorn Zachary,CFO,s,224.00,-150.0,9812.0,-2%,-33600.0,150.0,224.00,33600.0
2019-07-31,2019-07-29,TSLA,Straubel Jeffrey B,Chief Technical Officer,s,230.45,-15000.0,276398.0,-5%,-3456732.0,15000.0,230.45,3456732.0
2019-07-19,2019-07-17,TSLA,Kirkhorn Zachary,CFO,s,255.68,-150.0,9962.0,-1%,-38352.0,150.0,255.68,38352.0


In [81]:
insider_df.hvplot(y="Value_Pos", color="blue", title="All Insider Trades by Filing Date", ylabel="Value of Trades",rot=45).opts(yformatter='$%.0f')

:Curve   [index]   (Value_Pos)

In [82]:
# Filtering the TSLA Data
insider_df_msft = insider_df[insider_df["Ticker"]=="MSFT"]

#Verifying the company data
display(insider_df_msft.head())
display(insider_df_msft.tail())

,Trade Date,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value,Qty_Pos,Price_Pos,Value_Pos
2023-06-05,2023-06-02,MSFT,List Teri,Dir,s,334.77,-1250.0,1282.0,-49%,-418463.0,1250.0,334.77,418463.0
2023-05-19,2023-05-18,MSFT,Capossela Christopher C,"EVP, CMO",s,316.87,-5000.0,90632.0,-5%,-1584354.0,5000.0,316.87,1584354.0
2023-05-17,2023-05-16,MSFT,Capossela Christopher C,"EVP, CMO",s,312.54,-5081.0,95632.0,-5%,-1588024.0,5081.0,312.54,1588024.0
2023-05-10,2023-05-09,MSFT,Capossela Christopher C,"EVP, CMO",s,310.62,-9177.0,102333.0,-8%,-2850529.0,9177.0,310.62,2850529.0
2023-05-08,2023-05-05,MSFT,Capossela Christopher C,"EVP, CMO",s,310.05,-5000.0,111510.0,-4%,-1550250.0,5000.0,310.05,1550250.0


,Trade Date,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value,Qty_Pos,Price_Pos,Value_Pos
2019-08-14,2019-08-13,MSFT,Capossela Christopher C,"EVP, CMO",s,138.15,-7000.0,122817.0,-5%,-967050.0,7000.0,138.15,967050.0
2019-08-12,2019-08-08,MSFT,Capossela Christopher C,"EVP, CMO",s,138.00,-3500.0,129817.0,-3%,-483006.0,3500.0,138.00,483006.0
2019-08-05,2019-08-01,MSFT,Capossela Christopher C,"EVP, CMO",s,138.06,-3372.0,133317.0,-2%,-465542.0,3372.0,138.06,465542.0
2019-12-02,2019-08-01,MSFT,Capossela Christopher C,"EVP, CMO",s,138.06,-3372.0,133317.0,-2%,-465542.0,3372.0,138.06,465542.0
2019-07-30,2019-07-29,MSFT,Brod Frank H,Chief Accounting Officer,s,141.05,-4000.0,64152.0,-6%,-564216.0,4000.0,141.05,564216.0


In [83]:
insider_df_msft.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64 entries, 2023-06-05 to 2019-07-30
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Trade Date    64 non-null     object 
 1   Ticker        64 non-null     object 
 2   Insider Name  64 non-null     object 
 3   Title         64 non-null     object 
 4   Trade Type    64 non-null     object 
 5   Price         64 non-null     float64
 6   Qty           64 non-null     float64
 7   Owned         64 non-null     float64
 8   ΔOwn          64 non-null     object 
 9   Value         64 non-null     float64
 10  Qty_Pos       64 non-null     float64
 11  Price_Pos     64 non-null     float64
 12  Value_Pos     64 non-null     float64
dtypes: float64(7), object(6)
memory usage: 7.0+ KB


In [84]:
insider_df_msft.hvplot.bar(x="Title", y="Value_Pos", color="gold", title="MSFT Insider Trades", ylabel="Value of Trades",rot=45).opts(yformatter='$%.0f')

:Bars   [Title]   (Value_Pos)

In [85]:
insider_df_msft.hvplot(x="Trade Date", y="Value_Pos", color="blue", title="MSFT Insider Trades by Date", ylabel="Value of Trades",rot=90).opts(yformatter='$%.0f')

:Curve   [Trade Date]   (Value_Pos)

In [86]:
insider_df_msft_plot = insider_df_msft.hvplot.scatter(y="Price", color="orange", title="MSFT Insider Trades by Price", ylabel="Stock Price",rot=45, width=1000).opts(yformatter='$%.0f')
insider_df_msft_plot 

:Scatter   [index]   (Price)

In [87]:

import yfinance as yf

# Define the start and end dates for the historical data
start_date = "2019-06-28"
end_date = "2023-06-28"
ticker = "MSFT"

# Get the stock closing prices and volume for the specified time period
msft_stock = yf.Ticker(ticker)
msft_stock_data_df = msft_stock.history(start=start_date, end=end_date, interval="1d")

msft_stock_data_df = msft_stock_data_df.drop(["Open","High","Low","Stock Splits", "Dividends"], axis=1)

msft_stock_data_df.index = msft_stock_data_df.index.date



In [88]:
msft_stock_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1006 entries, 2019-06-28 to 2023-06-27
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   1006 non-null   float64
 1   Volume  1006 non-null   int64  
dtypes: float64(1), int64(1)
memory usage: 23.6+ KB


In [89]:
df_msft_stock_plot = msft_stock_data_df.hvplot(y="Close", color="blue", title="MSFT Stock Price", ylabel="Closing Price",rot=45, width=1000).opts(yformatter='$%.0f')
df_msft_stock_plot

:Curve   [index]   (Close)

In [90]:
overlay_msft_Plot = insider_df_msft_plot * df_msft_stock_plot
overlay_msft_Plot

:Overlay
   .Scatter.I :Scatter   [index]   (Price)
   .Curve.I   :Curve   [index]   (Close)

In [91]:
# Create the Moving averages and getting the closing price for file date.
msft_stock_data_df["MVA180"] = msft_stock_data_df["Close"].rolling(window=180).mean()
msft_stock_data_df["MVA20"] = msft_stock_data_df["Close"].rolling(window=20).mean()


insider_df_msft["ClosePrice"] = msft_stock_data_df["Close"]
insider_df_msft["MVA180"] = msft_stock_data_df["MVA180"]
insider_df_msft["MVA20"] = msft_stock_data_df["MVA20"]

display(msft_stock_data_df.head(10))
display(insider_df_msft.head(25))
display(msft_stock_data_df.tail(25))

/var/folders/kb/xkf419255xx2rp_mmzmhfwsh0000gn/T/ipykernel_51726/1246945398.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insider_df_msft["ClosePrice"] = msft_stock_data_df["Close"]
/var/folders/kb/xkf419255xx2rp_mmzmhfwsh0000gn/T/ipykernel_51726/1246945398.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insider_df_msft["MVA180"] = msft_stock_data_df["MVA180"]
/var/folders/kb/xkf419255xx2rp_mmzmhfwsh0000gn/T/ipykernel_51726/1246945398.py:8: SettingWithCopyWarning: 
A value is trying to be set on a

,Close,Volume,MVA180,MVA20
2019-06-28,128.756470,30043000,NaN,NaN
2019-07-01,130.409668,22613500,NaN,NaN
2019-07-02,131.274704,15237800,NaN,NaN
2019-07-03,132.120529,13629300,NaN,NaN
2019-07-05,131.736023,18141100,NaN,NaN
2019-07-08,131.639954,16779700,NaN,NaN
2019-07-09,131.159363,19953100,NaN,NaN
2019-07-10,132.495377,24204400,NaN,NaN
2019-07-11,133.024002,22327900,NaN,NaN
2019-07-12,133.504593,18936800,NaN,NaN


,Trade Date,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value,Qty_Pos,Price_Pos,Value_Pos,ClosePrice,MVA180,MVA20
2023-06-05,2023-06-02,MSFT,List Teri,Dir,s,334.77,-1250.0,1282.0,-49%,-418463.0,1250.0,334.77,418463.0,335.940002,259.955242,319.349521
2023-05-19,2023-05-18,MSFT,Capossela Christopher C,"EVP, CMO",s,316.87,-5000.0,90632.0,-5%,-1584354.0,5000.0,316.87,1584354.0,318.339996,255.928690,305.195753
2023-05-17,2023-05-16,MSFT,Capossela Christopher C,"EVP, CMO",s,312.54,-5081.0,95632.0,-5%,-1588024.0,5081.0,312.54,1588024.0,314.000000,255.282366,301.883884
2023-05-10,2023-05-09,MSFT,Capossela Christopher C,"EVP, CMO",s,310.62,-9177.0,102333.0,-8%,-2850529.0,9177.0,310.62,2850529.0,311.628754,254.185875,296.227930
2023-05-08,2023-05-05,MSFT,Capossela Christopher C,"EVP, CMO",s,310.05,-5000.0,111510.0,-4%,-1550250.0,5000.0,310.05,1550250.0,307.976746,253.862140,293.584207
2023-05-02,2023-05-01,MSFT,Althoff Judson,"EVP, Chief Commercial Officer",s,306.09,-30000.0,117456.0,-20%,-9182700.0,30000.0,306.09,9182700.0,304.743805,253.480388,289.762560
2023-03-02,2023-02-28,MSFT,Nadella Satya,CEO,s,248.88,-4767.0,788625.0,-1%,-1186411.0,4767.0,248.88,1186411.0,250.562256,249.723249,258.081977
2023-02-15,2023-02-13,MSFT,Capossela Christopher C,"EVP, CMO",s,272.32,-1000.0,117666.0,-1%,-272323.0,1000.0,272.32,272323.0,268.732544,250.363732,253.361385
2023-02-13,2023-02-09,MSFT,Capossela Christopher C,"EVP, CMO",s,269.38,-1000.0,118666.0,-1%,-269380.0,1000.0,269.38,269380.0,270.051758,250.332332,250.076581
2022-12-02,2022-12-01,MSFT,Althoff Judson,"EVP, Chief Commercial Officer",s,254.27,-24144.0,150047.0,-14%,-6139124.0,24144.0,254.27,6139124.0,253.827957,260.127993,240.107262


,Close,Volume,MVA180,MVA20
2023-05-22,321.179993,24115700,256.277166,307.196985
2023-05-23,315.260010,30797200,256.616683,309.219023
2023-05-24,313.850006,23384900,256.963862,310.175238
2023-05-25,325.920013,43301700,257.351408,311.262985
2023-05-26,332.890015,36630600,257.775305,312.577997
2023-05-30,331.209991,29503100,258.157116,313.893822
2023-05-31,328.390015,45950600,258.511184,315.076132
2023-06-01,332.579987,26773900,258.969365,316.518330
2023-06-02,335.399994,25864000,259.441945,318.051140
2023-06-05,335.940002,21307100,259.955242,319.349521


In [92]:
insider_df_msft.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64 entries, 2023-06-05 to 2019-07-30
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Trade Date    64 non-null     object 
 1   Ticker        64 non-null     object 
 2   Insider Name  64 non-null     object 
 3   Title         64 non-null     object 
 4   Trade Type    64 non-null     object 
 5   Price         64 non-null     float64
 6   Qty           64 non-null     float64
 7   Owned         64 non-null     float64
 8   ΔOwn          64 non-null     object 
 9   Value         64 non-null     float64
 10  Qty_Pos       64 non-null     float64
 11  Price_Pos     64 non-null     float64
 12  Value_Pos     64 non-null     float64
 13  ClosePrice    64 non-null     float64
 14  MVA180        42 non-null     float64
 15  MVA20         64 non-null     float64
dtypes: float64(10), object(6)
memory usage: 8.5+ KB


In [106]:
# Buy or Sell decision based on the 5 day moving average.

insider_df_msft.loc[insider_df_msft["MVA20"] > insider_df_msft["ClosePrice"],"Decision"]=1 # Buy decision

insider_df_msft.loc[insider_df_msft["MVA20"] <= insider_df_msft["ClosePrice"],"Decision"]=0 #Sell decision

insider_df_msft


,Trade Date,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value,Qty_Pos,Price_Pos,Value_Pos,ClosePrice,MVA180,MVA20,Decision
2023-06-05,2023-06-02,MSFT,List Teri,Dir,s,334.77,-1250.0,1282.0,-49%,-418463.0,1250.0,334.77,418463.0,335.940002,259.955242,319.349521,0.0
2023-05-19,2023-05-18,MSFT,Capossela Christopher C,"EVP, CMO",s,316.87,-5000.0,90632.0,-5%,-1584354.0,5000.0,316.87,1584354.0,318.339996,255.928690,305.195753,0.0
2023-05-17,2023-05-16,MSFT,Capossela Christopher C,"EVP, CMO",s,312.54,-5081.0,95632.0,-5%,-1588024.0,5081.0,312.54,1588024.0,314.000000,255.282366,301.883884,0.0
2023-05-10,2023-05-09,MSFT,Capossela Christopher C,"EVP, CMO",s,310.62,-9177.0,102333.0,-8%,-2850529.0,9177.0,310.62,2850529.0,311.628754,254.185875,296.227930,0.0
2023-05-08,2023-05-05,MSFT,Capossela Christopher C,"EVP, CMO",s,310.05,-5000.0,111510.0,-4%,-1550250.0,5000.0,310.05,1550250.0,307.976746,253.862140,293.584207,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-14,2019-08-13,MSFT,Capossela Christopher C,"EVP, CMO",s,138.15,-7000.0,122817.0,-5%,-967050.0,7000.0,138.15,967050.0,129.204498,NaN,132.313526,1.0
2019-08-12,2019-08-08,MSFT,Capossela Christopher C,"EVP, CMO",s,138.00,-3500.0,129817.0,-3%,-483006.0,3500.0,138.00,483006.0,130.515381,NaN,132.329090,1.0
2019-08-05,2019-08-01,MSFT,Capossela Christopher C,"EVP, CMO",s,138.06,-3372.0,133317.0,-2%,-465542.0,3372.0,138.06,465542.0,127.074471,NaN,132.720761,1.0
2019-12-02,2019-08-01,MSFT,Capossela Christopher C,"EVP, CMO",s,138.06,-3372.0,133317.0,-2%,-465542.0,3372.0,138.06,465542.0,144.710297,NaN,143.295742,0.0


In [94]:
insider_df_msft["Decision"].value_counts()

0.0    41
1.0    23
Name: Decision, dtype: int64

In [95]:
# Import modules for Logistic Regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


# Create X, or features DataFrame
features = insider_df_msft[['Price', 'Qty_Pos','Value_Pos','ClosePrice','MVA20']]

# Create y, or target DataFrame
target = insider_df_msft['Decision']

# Use train_test_split to separate the data
training_features, testing_features, training_targets, testing_targets = train_test_split(features, target, test_size=.4)


In [96]:
#Scale the data

scaler = StandardScaler()

X_scaler = scaler.fit(training_features)
training_features_scaled = X_scaler.transform(training_features)
testing_features_scaled = X_scaler.transform(testing_features)



In [97]:
#Instantiate a model
logistic_regression_model = LogisticRegression(random_state=7)

In [98]:
# Fit the model
logistic_regression_model.fit(training_features_scaled, training_targets)

LogisticRegression(random_state=7)

In [99]:
# Generate predictions from the model we just fit
predictions = logistic_regression_model.predict(training_features_scaled)

# Convert those predictions (and actual values) to a DataFrame
results_df = pd.DataFrame({"Prediction": predictions, "Actual": training_targets})
results_df

,Prediction,Actual
2019-11-05,0.0,0.0
2021-03-03,1.0,1.0
2019-08-12,0.0,1.0
2023-06-05,0.0,0.0
2020-03-04,0.0,1.0
2022-09-13,1.0,1.0
2020-09-04,1.0,1.0
2019-07-30,0.0,0.0
2020-09-02,0.0,0.0
2020-05-04,0.0,0.0


In [100]:
# Apply the fitted model to the test dataset
testing_predictions = logistic_regression_model.predict(testing_features_scaled)

# Save both the test predictions and actual test values to a DataFrame
results_df = pd.DataFrame({
    "Testing Data Predictions": testing_predictions,
    "Testing Data Actual Targets": testing_targets})
results_df

,Testing Data Predictions,Testing Data Actual Targets
2019-12-02,0.0,0.0
2020-02-04,0.0,0.0
2023-02-15,0.0,0.0
2023-05-17,0.0,0.0
2023-02-13,0.0,0.0
2020-09-09,0.0,1.0
2021-11-24,1.0,0.0
2022-02-09,0.0,0.0
2020-11-06,1.0,0.0
2019-09-06,0.0,0.0


In [101]:
# Calculate the model's accuracy on the test dataset
accuracy_score(testing_targets, testing_predictions)

0.7307692307692307

In [102]:
#Print the confusion matrix for training data
training_matrix = confusion_matrix(training_targets, predictions)
print(training_matrix)

[[16  3]
 [ 9 10]]


In [103]:
#Print the confusion matrix for testing data
testing_matrix = confusion_matrix(testing_targets, testing_predictions)
print(testing_matrix)

[[18  4]
 [ 3  1]]


In [104]:
# Classification report for training data
training_report = classification_report(training_targets, predictions)
# Then view its output
print(training_report)

              precision    recall  f1-score   support

         0.0       0.64      0.84      0.73        19
         1.0       0.77      0.53      0.62        19

    accuracy                           0.68        38
   macro avg       0.70      0.68      0.68        38
weighted avg       0.70      0.68      0.68        38



In [105]:
# Classification report for testing data
testing_report = classification_report(testing_targets, testing_predictions)
# Then view its output
print(testing_report)

              precision    recall  f1-score   support

         0.0       0.86      0.82      0.84        22
         1.0       0.20      0.25      0.22         4

    accuracy                           0.73        26
   macro avg       0.53      0.53      0.53        26
weighted avg       0.76      0.73      0.74        26

